# Equipment_move_workflow

# IT_Equipment_unique_records

Workflow permettant la mise à jour de la table `IT_equipments_unique_equipments`

`IT_Equipment_unique_records` : table regroupant tous les asset_id unique, une ligne par equipments avec tous les attribus prédifinis auquel on rajoute : 
    
    - First_occurence : première d'ajout d'un fichier
    - last_move_date : date du dernier changement de position
    - last_move_date : dernier changement de position
    - last_move_salle : Salle du dernier changement de position
    - Clean date : Si un equipment est jugé clean, la date du dernier ajout est affiché 

In [ ]:
drop table if exists `IT_Equipment_unique_records` ;
create table `IT_Equipment_unique_records`
as 
select * from `IT_Equipment_records`
GROUP BY ID_Systeme_Asset ;

In [ ]:
ALTER TABLE IT_Equipment_unique_records
ADD COLUMN first_occurence varchar(255),
ADD COLUMN last_move_salle varchar(255),
ADD COLUMN last_move varchar(255),
ADD COLUMN last_move_date varchar(255),
ADD COLUMN clean_date varchar(255) ;

`IT_Equipment_sorted_by_date_asc` : table ordonnée par date croissante 

`IT_Equipment_sorted_by_date_desc` : table ordonnée par date décroissante

In [ ]:
drop view if exists IT_Equipment_sorted_by_date_asc ;
create view `IT_Equipment_sorted_by_date_asc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') ASC ;

drop view if exists IT_Equipment_sorted_by_date_desc ;
create view `IT_Equipment_sorted_by_date_desc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC ;

`IT_Equipment_first_last_occurence_per_id_all` : 1er occurence et dernière occurence pour chaque asset_id

In [ ]:
drop view if exists IT_Equipment_first_last_occurence_per_id_all ;
CREATE view IT_Equipment_first_last_occurence_per_id_all
as 
select min(inserted) first_records, max(inserted) last_records, it.ID_Systeme_Asset from `IT_Equipment_sorted_by_date_asc` it
GROUP BY ID_Systeme_Asset
order by INSERTED desc ;

`IT_Equipment_first_occurence_per_id` : Première occurence (date d'ajout d'un equipment) 

`IT_Equipment_last_occurence_per_id` : Dernière occurence (dernière date d'update d'un equipment) 

In [ ]:
drop view if exists IT_Equipment_first_occurence_per_id ;
CREATE view  IT_Equipment_first_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.first_records ;

drop view if exists IT_Equipment_last_occurence_per_id ;
CREATE view  IT_Equipment_last_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.last_records ;

`IT_Equipment_last_occurence_position` : Asset, salle, position et date d'insertion de la dernière position connu d'un equipment. Exemple : changement de position de 22-06-01, et mise à jour le  22-06-03 & 22-06-10. La première fois que la nouvelle position apparait est affiché (donc 22-06-01)

In [ ]:
drop view if exists IT_Equipment_last_occurence_position ;
CREATE view  IT_Equipment_last_occurence_position
as
SELECT it2.ID_Systeme_Asset, it2.nom_de_la_salle, it2.Reperage_ID, it2.INSERTED
FROM `IT_Equipment_last_occurence_per_id` it
INNER JOIN `IT_Equipment_records` it2 ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset AND it.reperage_id = it2.reperage_id
GROUP BY it2.ID_Systeme_Asset

`last_date_insertion` : dernière date du fichier csv de mise à jour de la base. Exemple IT_equipment.01.06.22 -> 22-06-01

In [ ]:
drop view if EXISTS last_date_insertion ;
create view last_date_insertion
as
select DISTINCT(INSERTED) source_insert from `IT_Equipment_records`
ORDER by INSERTED desc
limit 1

`date_insertion_records` : table historique de toutes les mise à jour de la base

In [ ]:
drop view if EXISTS date_insertion_records ; 
create view date_insertion_records
as
select DISTINCT(INSERTED) source_insert from `IT_Equipment_records`
ORDER by INSERTED desc ;

`IT_Equipment_cleaned_equipment` : Table des equipments clean, avec leur asset_id, leur dernière position connue et la dernière date d'apparition dans un fichier csv. Un Equipement est jugé clean s'il n'apparait pas dans la dernière MàJ de la base

In [ ]:
drop view if EXISTS IT_Equipment_cleaned_equipment;
create view IT_Equipment_cleaned_equipment
as 
select it.nom, it.ID_Systeme_Asset, it.nom_de_la_salle, it.Reperage_ID,it.SOURCES, it.INSERTED last_insert_date  from `IT_Equipment_last_occurence_per_id` it
LEFT JOIN `last_date_insertion` dat ON dat.source_insert = it.inserted
where dat.source_insert is null ;

## Update de la table `IT_Equipment_unique_records`

### First occurence

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set first_occurence = it2.inserted

### Last move

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_last_occurence_position` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set last_move_date = it2.INSERTED ,
    last_move = it2.Reperage_ID ,
    last_move_salle = it2.nom_de_la_salle

### Clean date

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_cleaned_equipment` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set clean_date = it2.last_insert_date
